### How to Use:
1. **Replace API Keys:** Update `YOUR_GROQ_API_KEY`, `YOUR_TWILIO_SID`, etc.
2. **Run the Notebook:** Save this as `grocery_price_bot.ipynb` and execute it.
3. **Start the Flask Server:** Run `python grocery_price_bot.py`
4. **Set Up Twilio Webhook:** Use `ngrok` to expose your Flask app.
5. **Message WhatsApp Bot:** Send an item name (e.g., `"Bread"`) to get the best prices.

🚀 **Now you're ready to compare grocery prices like a pro!** 🚀

In [ ]:
```python
# Grocery Price Comparison using Groq Vision & Twilio
# ---------------------------------------------------
# This notebook extracts grocery flyer data using Groq's Llama-3.2-11b-Vision-Preview model,
# stores it in a database, and allows price lookup via a Twilio WhatsApp chatbot.

import os
import re
import pandas as pd
import sqlite3
import requests
import json
from openai import OpenAI
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse

# Set up API Keys (Replace with your actual keys)
GROQ_API_KEY = "YOUR_GROQ_API_KEY"
TWILIO_SID = "YOUR_TWILIO_SID"
TWILIO_AUTH_TOKEN = "YOUR_TWILIO_AUTH_TOKEN"
TWILIO_WHATSAPP_NUMBER = "YOUR_TWILIO_WHATSAPP_NUMBER"

# Function to extract flyer data using Groq Vision Model
def extract_flyer_data(image_path):
    client = OpenAI(api_key=GROQ_API_KEY, base_url="https://api.groq.com/openai/v1")

    with open(image_path, "rb") as image:
        response = client.chat.completions.create(
            model="llama-3.2-11b-vision-preview",
            messages=[
                {
                    "role": "system",
                    "content": "Extract the item names and their corresponding prices from the flyer. Provide the data in a structured format like a table. The column name should be 'Item', 'Quantity'(like kg, oz, ml, l, etc. Think from what you see and put the quantity with its unit accordingly, 'Price'(everything is in $, so do not include that sign, just put the number. Also, it is mostly produce so the items would be in the price range of 1.99, 2.99, 3.99, etc or 2 digits, Moreover, put 1 item in 1 row, 'Notes' everything else that you think is relevant should be in notes. Like Sale, or anything extra that you see. In the end, provide a dataframe.))"
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Process this flyer and return structured tabular data."},
                        {"type": "image", "image": image.read()}
                    ]
                }
            ],
            max_tokens=1000
        )

    response_text = response.choices[0].message.content
    return response_text

# Function to convert extracted text into a DataFrame
def parse_flyer_data(response_text):
    lines = response_text.strip().split("\n")[2:]  # Skip header
    data = [line.split("|")[1:-1] for line in lines]  # Extract columns

    df = pd.DataFrame(data, columns=["Item", "Quantity", "Price", "Notes"])
    df["Price"] = df["Price"].apply(lambda x: float(re.sub(r"[^\d.]", "", x)) if x.strip() else None)
    return df

# Process flyers for Freshco and No Frills
df_freshco = parse_flyer_data(extract_flyer_data("freshco_flyer.jpg"))
df_nofrills = parse_flyer_data(extract_flyer_data("nofrills_flyer.jpg"))

# Store DataFrames in SQLite Database
conn = sqlite3.connect("grocery_flyers.db")
df_freshco.to_sql("freshco", conn, if_exists="replace", index=False)
df_nofrills.to_sql("nofrills", conn, if_exists="replace", index=False)

print("Data stored successfully in SQLite!")

# Function to find the best price for an item
def find_best_price(item_name):
    query = f"""
        SELECT 'Freshco' AS Store, Item, Price FROM freshco WHERE Item LIKE '%{item_name}%'
        UNION ALL
        SELECT 'No Frills' AS Store, Item, Price FROM nofrills WHERE Item LIKE '%{item_name}%'
        ORDER BY Price ASC
    """
    return pd.read_sql_query(query, conn)

# Flask app for Twilio WhatsApp integration
app = Flask(__name__)

@app.route("/whatsapp", methods=["POST"])
def whatsapp_bot():
    incoming_msg = request.values.get("Body", "").strip().lower()
    response = MessagingResponse()
    message = response.message()

    if incoming_msg:
        df = find_best_price(incoming_msg)
        if not df.empty:
            result = df.to_string(index=False)
            message.body(f"Here are the best prices for '{incoming_msg}':\n\n{result}")
        else:
            message.body(f"Sorry, no matching items found for '{incoming_msg}'.")

    return str(response)

# Run Flask Server (for local testing, use ngrok to expose port 5000)
if __name__ == "__main__":
    app.run(port=5000)
```

